# Query book

In [ ]:
import lamindb as ln
import lamindb.schema as lns

from sqlmodel import or_

ln.Run()

## Basic select operations

### Output formats

In [ ]:
ln.select(ln.DObject).df()

In [ ]:
ln.select(ln.DObject).all()

In [ ]:
ln.select(ln.DObject).first()

In [ ]:
ln.select(ln.DObject, name="Mouse Lymph Node scRNA-seq", suffix=".h5ad").one()

### And

In [ ]:
ln.select(ln.DObject).where(
    ln.DObject.name == "Mouse Lymph Node scRNA-seq", ln.DObject.suffix == ".h5ad"
).one()

### Or

In [ ]:
ln.select(ln.DObject).where(
    or_(ln.DObject.suffix == ".zarr", ln.DObject.suffix == ".h5ad")
).df()

### In

In [ ]:
ln.select(ln.DObject).where(ln.DObject.suffix.in_([".zarr", ".h5ad"])).df()

### Order by

In [ ]:
ln.select(lns.Run).order_by(lns.Run.created_at.desc()).df()

### Contains

In [ ]:
ln.select(lns.Notebook).where(lns.Notebook.title.contains("Link")).df()

## Data lineage

### Track run, inputs & outputs

Which run does this file `output.bam` come from?

In [ ]:
ss = ln.Session()

run = ss.select(lns.Run).join(ln.DObject, name="output", suffix=".bam").one()

In [ ]:
run.inputs

In [ ]:
run.outputs

In [ ]:
ss.close()

### Track notebook

Which notebook ingested this dataset `Mouse Lymph Node scRNA-seq.h5ad`?

In [ ]:
ln.select(lns.Notebook).join(lns.Run).join(
    ln.DObject, name="Mouse Lymph Node scRNA-seq", suffix=".h5ad"
).one()

Which notebooks are produced by testuser2?

In [ ]:
ln.select(lns.Notebook).join(lns.User, handle="testuser2").df()

### Track pipeline

Which pipeline produced this file `input.fastq.gz`?

In [ ]:
with ln.Session() as ss:
    dobject = ss.select(ln.DObject, name="input", suffix=".fastq.gz").one()
    print(dobject.source.pipeline)

Which pipelines are registered by testuser1?

In [ ]:
ln.select(lns.Pipeline).join(lns.User, handle="testuser1").df()

### Track user

Which users have interacted with the database?

In [ ]:
ln.select(lns.User).df()

Which user ingested this file `input.fastq.gz`?

In [ ]:
ln.select(lns.User).join(lns.Run).join(
    ln.DObject, name="input", suffix=".fastq.gz"
).one()

Which users created notebooks with "lineage" in the title?

In [ ]:
ln.select(lns.User).join(lns.Notebook).where(
    lns.Notebook.title.contains("lineage")
).df()

Which user registered this pipeline?

In [ ]:
ln.select(lns.User).join(lns.Pipeline, name="Cell Ranger", v="7").one()